# Forecasting Consensus Expectations: Consumer Price Index

## Point + Directional + Distributional Forecasts

In [30]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
import seaborn as sns
import matplotlib.dates as mdates
import statsmodels.api as sm
import plotly.express as px

from tqdm.auto import tqdm
from scipy import stats, special
from scipy.optimize import brentq
from collections import defaultdict
from itertools import product
from scipy.stats import t as student_t, norm, binomtest, jarque_bera
from statsmodels.stats.diagnostic import het_breuschpagan

In [31]:
OUT_DIR = "../out"        
MOM_DF_FILE       = "cpi_mom_df.parquet"
MOM_DF_FULL_FILE  = "cpi_mom_df_full.parquet"
YOY_DF_FILE       = "cpi_yoy_df.parquet"
YOY_DF_FULL_FILE  = "cpi_yoy_df_full.parquet"

mom_df       = pd.read_parquet(os.path.join(OUT_DIR, MOM_DF_FILE),      engine="pyarrow")
mom_df_full  = pd.read_parquet(os.path.join(OUT_DIR, MOM_DF_FULL_FILE), engine="pyarrow")

yoy_df       = pd.read_parquet(os.path.join(OUT_DIR, YOY_DF_FILE),      engine="pyarrow")
yoy_df_full  = pd.read_parquet(os.path.join(OUT_DIR, YOY_DF_FULL_FILE), engine="pyarrow")

print("mom_df shape     :", mom_df.shape)
print("mom_df_full shape:", mom_df_full.shape)

print("yoy_df shape     :", yoy_df.shape)
print("yoy_df_full shape:", yoy_df_full.shape)

mom_df shape     : (50347, 11)
mom_df_full shape: (59455, 11)
yoy_df shape     : (32238, 11)
yoy_df_full shape: (38070, 11)


In [32]:
mom_df.head()

,release_date,period,median_survey,actual,economist,firm,forecast,asof,error,surprise,series
0,2006-01-18,2005-12-31,0.2000,0.2000,Adam Chester,Lloyds Bank PLC,NaN,NaT,NaN,0.0000,Core CPI M/M
1,2006-01-18,2005-12-31,0.2000,0.2000,Alessandro Truppia,Aletti Gestielle Sgr Spa,NaN,NaT,NaN,0.0000,Core CPI M/M
2,2006-01-18,2005-12-31,0.2000,0.2000,Alison Lynn Reaser,Point Loma Nazarene University,0.2000,2006-01-16,0.0000,0.0000,Core CPI M/M
3,2006-01-18,2005-12-31,0.2000,0.2000,Allan Von Mehren,Danske Bank AS,NaN,NaT,NaN,0.0000,Core CPI M/M
4,2006-01-18,2005-12-31,0.2000,0.2000,Andreas Busch,Bantleon AG,0.2000,2006-01-16,0.0000,0.0000,Core CPI M/M


In [33]:
yoy_df.head()

,release_date,period,median_survey,actual,economist,firm,forecast,asof,error,surprise,series
0,2006-01-18,2005-12-31,2.2000,2.2000,Adam Chester,Lloyds Bank PLC,NaN,NaT,NaN,0.0000,Core CPI Y/Y
1,2006-01-18,2005-12-31,2.2000,2.2000,Allan Von Mehren,Danske Bank AS,NaN,NaT,NaN,0.0000,Core CPI Y/Y
2,2006-01-18,2005-12-31,2.2000,2.2000,Andreas Busch,Bantleon AG,NaN,NaT,NaN,0.0000,Core CPI Y/Y
3,2006-01-18,2005-12-31,2.2000,2.2000,Andrew Gretzinger,Manulife Asset Management Limited,NaN,NaT,NaN,0.0000,Core CPI Y/Y
4,2006-01-18,2005-12-31,2.2000,2.2000,Aneta Markowska,Moore Capital Management LP,NaN,NaT,NaN,0.0000,Core CPI Y/Y


In [34]:
import pandas as pd
import numpy as np

# ------------------------------------------------------------------
# Helper:  count decimal places in a numeric Series
# ------------------------------------------------------------------
def count_decimals(series: pd.Series, max_scan: int = 1_000_000):
    """
    Return a value-counts Series telling you how many observations have
    0, 1, 2, … decimal places. NaNs are ignored.

    Parameters
    ----------
    series : pd.Series
        The numeric column to inspect.
    max_scan : int, optional
        Safety cap on how many rows to scan (defaults to 1 million).

    Examples
    --------
    >>> count_decimals(df['forecast'])
    0    378
    1    112
    2     45
    dtype: int64
    """
    # work on a copy of numeric values only
    vals = series.dropna().astype(float).to_numpy()[:max_scan]

    def decimals_of(x: float) -> int:
        # Convert to string, strip trailing zeros and the dot itself
        s = format(x, ".10f").rstrip("0").rstrip(".")
        return len(s.partition(".")[2])

    dec_counts = pd.Series([decimals_of(v) for v in vals]).value_counts().sort_index()
    dec_counts.index.name = "# decimal places"
    dec_counts.name = "count"
    return dec_counts

# ------------------------------------------------------------------
# Run on each CPI dataframe / column you care about
# ------------------------------------------------------------------
PANELS = {
    "Core CPI M/M  – forecast"      : mom_df["forecast"],
    "Core CPI M/M  – median_survey" : mom_df["median_survey"],
    "Core CPI Y/Y  – forecast"      : yoy_df["forecast"],
    "Core CPI Y/Y  – median_survey" : yoy_df["median_survey"],
}

for label, col in PANELS.items():
    print(f"\n{label}")
    print(count_decimals(col))



Core CPI M/M  – forecast
# decimal places
0      342
1    12932
2      400
3       81
4       13
5        1
Name: count, dtype: int64

Core CPI M/M  – median_survey
# decimal places
1    49335
2     1012
Name: count, dtype: int64

Core CPI Y/Y  – forecast
# decimal places
0     666
1    6442
2      69
3      48
4      11
Name: count, dtype: int64

Core CPI Y/Y  – median_survey
# decimal places
0     2754
1    28674
2      486
3      324
Name: count, dtype: int64


In [35]:
import pandas as pd
import numpy as np

# ------------------------------------------------------------
# helper: how many decimals does a single number have?
# ------------------------------------------------------------
def _decimals(x: float) -> int:
    """
    Return the number of digits after the decimal point for *one* float.
    NaNs → -1  (so they never match our >=4 filter)
    """
    if pd.isna(x):
        return -1
    s = format(float(x), ".10f").rstrip("0").rstrip(".")
    return len(s.partition(".")[2])

# ------------------------------------------------------------
# main inspection function
# ------------------------------------------------------------
def show_high_precision(df: pd.DataFrame,
                        col: str = "forecast",
                        min_decimals: int = 4):
    """
    Display all rows where `col` has ≥ `min_decimals` decimal places.

    Parameters
    ----------
    df : pd.DataFrame
    col : str
        Name of the numeric column to inspect (default "forecast").
    min_decimals : int
        Threshold of decimal places to flag (default 4).

    Returns
    -------
    pd.DataFrame
        The filtered rows, ready for further inspection.
    """
    mask = df[col].apply(_decimals) >= min_decimals
    flagged = df.loc[mask].copy()

    print(f"→ {len(flagged)} rows in '{col}' with ≥ {min_decimals} decimals\n")
    display_cols = ["release_date", "economist", "firm", col, "asof"]
    display(flagged[display_cols].sort_values("release_date").head(20))  # show first 20
    return flagged

# ------------------------------------------------------------
# EXAMPLE USAGE
# ------------------------------------------------------------
# For Core CPI M/M:
high_prec_mom = show_high_precision(mom_df)

# For Core CPI Y/Y:
high_prec_yoy = show_high_precision(yoy_df)


→ 14 rows in 'forecast' with ≥ 4 decimals



,release_date,economist,firm,forecast,asof
34531,2017-05-12,Kevin Cummins,Natwest Markets,0.2211,2017-05-08
46911,2024-06-12,John D Herrmann,Herrmann Forecasting LLC,0.2591,2024-06-12
47164,2024-07-11,John D Herrmann,Herrmann Forecasting LLC,0.1892,2024-07-10
47417,2024-08-14,John D Herrmann,Herrmann Forecasting LLC,0.1888,2024-08-14
47670,2024-09-11,John D Herrmann,Herrmann Forecasting LLC,0.2474,2024-09-11
47923,2024-10-10,John D Herrmann,Herrmann Forecasting LLC,0.2305,2024-10-09
48176,2024-11-13,John D Herrmann,Herrmann Forecasting LLC,0.2298,2024-11-13
48429,2024-12-11,John D Herrmann,Herrmann Forecasting LLC,0.2605,2024-12-10
48682,2025-01-15,John D Herrmann,Herrmann Forecasting LLC,0.2712,2025-01-14
48935,2025-02-12,John D Herrmann,Herrmann Forecasting LLC,0.3037,2025-02-11


→ 11 rows in 'forecast' with ≥ 4 decimals



,release_date,economist,firm,forecast,asof
30046,2024-06-12,John D Herrmann,Herrmann Forecasting LLC,3.5102,2024-06-12
30208,2024-07-11,John D Herrmann,Herrmann Forecasting LLC,3.4054,2024-07-10
30370,2024-08-14,John D Herrmann,Herrmann Forecasting LLC,3.2375,2024-08-14
30694,2024-10-10,John D Herrmann,Herrmann Forecasting LLC,3.1744,2024-10-09
31018,2024-12-11,John D Herrmann,Herrmann Forecasting LLC,3.2512,2024-12-10
31180,2025-01-15,John D Herrmann,Herrmann Forecasting LLC,3.2959,2025-01-14
31342,2025-02-12,John D Herrmann,Herrmann Forecasting LLC,3.1571,2025-02-11
31504,2025-03-12,John D Herrmann,Herrmann Forecasting LLC,3.2112,2025-03-11
31666,2025-04-10,John D Herrmann,Herrmann Forecasting LLC,3.1165,2025-04-09
31828,2025-05-13,John D Herrmann,Herrmann Forecasting LLC,2.9065,2025-05-12


## Static inverse-MSE

In [36]:
# ================================================================
# 3-class grid search  +  confusion matrix w/ percentages + live call
# ================================================================
import numpy as np, pandas as pd
from tqdm.auto import tqdm

# ---------------- CONFIG -----------------
TOLS     = [0.001, 0.004, 0.006, 0.008, 0.01]          # ppt
WINDOWS  = [3, 6, 12]
METHODS  = ["inverse_mse", "inverse_mae", "equal_weight"]
RIDGE    = 1e-6
START15  = pd.Timestamp("2015-01-01")

PANELS = [("Core CPI M/M", mom_df_full),
          ("Core CPI Y/Y", yoy_df_full)]
# -----------------------------------------

def weights_for_method(hist, method):
    econs = hist["economist"].unique()
    if method == "equal_weight":
        return pd.Series(1.0 / len(econs), index=econs)
    err = hist.groupby("economist")["error"].apply(
        lambda s: np.nanmean((s**2) if method == "inverse_mse" else np.abs(s)))
    w = 1.0 / (err + RIDGE)
    return w / w.sum()

def smart_one_date(df, t, window, method):
    dates = np.sort(df["release_date"].unique())
    idx   = np.where(dates == t)[0][0]
    hist  = df[df["release_date"].isin(dates[idx-window:idx])]
    elig  = hist.groupby("economist")["forecast"].apply(lambda s: s.notna().all())
    econs = elig[elig].index
    if econs.empty:
        return np.nan
    w = weights_for_method(hist[hist["economist"].isin(econs)], method)
    cur = df[(df["release_date"] == t) & (df["economist"].isin(w.index))]
    f_t = cur.set_index("economist")["forecast"].dropna()
    w   = w.reindex(f_t.index).dropna()
    if w.empty:
        return np.nan
    w /= w.sum()
    return float(np.dot(w, f_t.loc[w.index]))

def cm_flat(y_true, y_pred):
    labs = ["miss", "hit", "beat"]
    cm = pd.crosstab(pd.Categorical(y_pred, labs),
                     pd.Categorical(y_true, labs)).reindex(index=labs,
                                                            columns=labs,
                                                            fill_value=0)
    return cm.values.flatten()          # 9-element row-major

# ---------------- main loop ----------------
rows, live_calls = [], []

for panel_name, df_full in tqdm(PANELS, desc="Panels"):
    df_full = df_full.copy()
    latest_date = df_full["release_date"].max()

    realised_df  = df_full[(df_full["actual"].notna()) &
                           (df_full["release_date"] >= START15)]
    unreleased_df = df_full[(df_full["release_date"] == latest_date) &
                            (df_full["actual"].isna())]

    for W, meth in tqdm([(w, m) for w in WINDOWS for m in METHODS],
                        desc=f"{panel_name} specs", leave=False):

        dates = np.sort(realised_df["release_date"].unique())
        temp  = []
        for idx in tqdm(range(W, len(dates)),
                        desc=f"{panel_name} {meth} {W}-mo",
                        leave=False, disable=len(dates) < 50):
            t = dates[idx]
            smart = smart_one_date(realised_df, t, W, meth)
            if np.isnan(smart):
                continue
            row = realised_df[realised_df["release_date"] == t].iloc[0]
            temp.append((smart, row["median_survey"], row["actual"]))
        if not temp:
            continue

        temp = pd.DataFrame(temp, columns=["smart", "median", "actual"])
        temp["d"]        = temp["smart"] - temp["median"]
        temp["true_cls"] = np.where(temp["actual"] < temp["median"], "miss",
                             np.where(temp["actual"] > temp["median"], "beat",
                                      "hit"))

        for tau in TOLS:
            pred_cls = np.where(temp["d"] < -tau, "miss",
                        np.where(temp["d"] >  tau, "beat", "hit"))
            hit_rate = (pred_cls == temp["true_cls"]).mean()
            cm_vec   = cm_flat(temp["true_cls"], pred_cls)

            rows.append({"panel"      : panel_name,
                         "window"     : W,
                         "method"     : meth,
                         "tau"        : tau,
                         "obs"        : len(temp),
                         "RMSE_smart" : np.sqrt(np.mean((temp["smart"]-temp["actual"])**2)),
                         "RMSE_median": np.sqrt(np.mean((temp["median"]-temp["actual"])**2)),
                         "HitRate"    : hit_rate,
                         **{f"CM_{i}": cm_vec[i] for i in range(9)}})

        # -------- live forecast -------
        if not unreleased_df.empty:
            smart_live = smart_one_date(df_full, latest_date, W, meth)
            if not np.isnan(smart_live):
                median_live = unreleased_df["median_survey"].iloc[0]
                sub = [r for r in rows if (r["panel"] == panel_name and
                                           r["window"] == W and
                                           r["method"] == meth)]
                best_tau = max(sub, key=lambda x: x["HitRate"])["tau"]
                d_live   = smart_live - median_live
                signal   = ("miss" if d_live < -best_tau else
                            "beat" if d_live >  best_tau else
                            "hit")
                live_calls.append({"panel":panel_name,"window":W,"method":meth,
                                   "tau":best_tau,"date":latest_date,
                                   "smart":smart_live,"median":median_live,
                                   "signal":signal})

# -------------- BACK-TEST OUTPUT -----------------
result_df = pd.DataFrame(rows).sort_values(
    ["panel","window","method","tau"]).reset_index(drop=True)

pd.set_option("display.float_format", "{:.4f}".format)
for p in result_df["panel"].unique():
    print(f"\n--- {p}  (2015-present realised) ---")
    print(result_df[result_df["panel"] == p]
          .loc[:, ["panel","window","method","tau",
                   "obs","RMSE_smart","RMSE_median","HitRate"]]
          .to_string(index=False))

    # ----- best spec & confusion matrix -----
    sub = result_df[result_df["panel"] == p].dropna(subset=["HitRate"])
    if sub.empty:
        print("\n(No valid specs for confusion matrix)\n")
        continue
    best = sub.loc[sub["HitRate"].idxmax()]
    print(f"\nSelected model: window={int(best['window'])} mo, "
          f"method={best['method']}, τ={best['tau']:.3f}")
    print(f"Back-test HitRate: {best['HitRate']:.3f}")

    cm = np.array([best[f"CM_{i}"] for i in range(9)]).reshape(3, 3)
    cm_df = pd.DataFrame(
        cm,
        index=["Pred miss","Pred hit","Pred beat"],
        columns=["Actual miss","Actual hit","Actual beat"])
    # per-class recall (diagonal / column total)
    col_tot = cm.sum(axis=0, keepdims=True)
    recall_pct = np.divide(np.diag(cm), col_tot.squeeze(),
                           out=np.zeros_like(np.diag(cm), dtype=float),
                           where=col_tot.squeeze()!=0)
    rec_series = pd.Series(recall_pct, index=["miss","hit","beat"])
    print("\nConfusion matrix (counts)")
    print(cm_df.to_string())
    print("\nPer-class recall (% correct of each actual class)")
    print((rec_series*100).round(1).astype(str) + '%')
    print("\n")

# -------------- LIVE FORECASTS -------------------
if live_calls:
    print("=== LIVE FORECASTS (latest unreleased print) ===")
    for call in live_calls:
        desc = {"beat":"Higher-than-median",
                "miss":"Lower-than-median",
                "hit":"On-median"}[call["signal"]]
        print(f"\n>>> {call['panel']}  •  {call['window']}-mo "
              f"{call['method']}  (τ={call['tau']:.3f})")
        print(f"Date   : {pd.to_datetime(call['date']).date()}")
        print(f"Smart  : {call['smart']:.4f}")
        print(f"Median : {call['median']:.4f}")
        print(f"Signal : {desc}")
else:
    print("\nNo unreleased CPI prints – all actuals available.")


Panels:   0%|          | 0/2 [00:00<?, ?it/s]

Core CPI M/M specs:   0%|          | 0/9 [00:00<?, ?it/s]

Core CPI M/M inverse_mse 3-mo:   0%|          | 0/123 [00:00<?, ?it/s]

Core CPI M/M inverse_mae 3-mo:   0%|          | 0/123 [00:00<?, ?it/s]

Core CPI M/M equal_weight 3-mo:   0%|          | 0/123 [00:00<?, ?it/s]

...

Core CPI Y/Y equal_weight 6-mo:   0%|          | 0/120 [00:00<?, ?it/s]

Core CPI Y/Y inverse_mse 12-mo:   0%|          | 0/114 [00:00<?, ?it/s]

Core CPI Y/Y inverse_mae 12-mo:   0%|          | 0/114 [00:00<?, ?it/s]

Core CPI Y/Y equal_weight 12-mo:   0%|          | 0/114 [00:00<?, ?it/s]


--- Core CPI M/M  (2015-present realised) ---
       panel  window       method    tau  obs  RMSE_smart  RMSE_median  HitRate
Core CPI M/M       3 equal_weight 0.0010  123      0.1312       0.1332   0.4390
Core CPI M/M       3 equal_weight 0.0040  123      0.1312       0.1332   0.4309
Core CPI M/M       3 equal_weight 0.0060  123      0.1312       0.1332   0.4228
Core CPI M/M       3 equal_weight 0.0080  123      0.1312       0.1332   0.4146
Core CPI M/M       3 equal_weight 0.0100  123      0.1312       0.1332   0.3902
Core CPI M/M       3  inverse_mae 0.0010  123      0.1306       0.1332   0.4553
Core CPI M/M       3  inverse_mae 0.0040  123      0.1306       0.1332   0.4634
Core CPI M/M       3  inverse_mae 0.0060  123      0.1306       0.1332   0.4553
Core CPI M/M       3  inverse_mae 0.0080  123      0.1306       0.1332   0.4472
Core CPI M/M       3  inverse_mae 0.0100  123      0.1306       0.1332   0.4472
Core CPI M/M       3  inverse_mse 0.0010  123      0.1301       0.1332   